In [2]:
%load_ext autoreload
%autoreload 2

# Tinygrid ERCOT API Demo

This notebook demonstrates how to use tinygrid to access ERCOT grid data with a clean, unified API.

**Features:**
- Type-safe enums for markets and location types
- Convenient date handling with keywords like "today" and "yesterday"
- Unified methods that route to the correct endpoints automatically
- Built-in location filtering
- Dashboard API for real-time grid data (no authentication required)
- Historical yearly data from ERCOT's MIS document system

## Setup

First, set your ERCOT API credentials as environment variables:

```bash
export ERCOT_USERNAME="your-email@example.com"
export ERCOT_PASSWORD="your-password"
export ERCOT_SUBSCRIPTION_KEY="your-subscription-key"
```

Or create a `.env` file in the examples directory.

In [3]:
import os

import pandas as pd
from dotenv import load_dotenv

from tinygrid import ERCOT, ERCOTAuth, ERCOTAuthConfig, LocationType, Market

# Load environment variables from .env file
load_dotenv()
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 200)

## Create Authenticated Client

In [4]:
# Create authenticated client
auth = ERCOTAuth(
    ERCOTAuthConfig(
        username=os.environ["ERCOT_USERNAME"],
        password=os.environ["ERCOT_PASSWORD"],
        subscription_key=os.environ["ERCOT_SUBSCRIPTION_KEY"],
    )
)

ercot = ERCOT(auth=auth)
print("Client created successfully!")

Client created successfully!


## Market and Location Types

Tinygrid uses enums for type safety and IDE autocomplete.

In [5]:
print("Available Markets:")
for m in Market:
    print(f"  Market.{m.name}")

print("\nAvailable Location Types:")
for lt in LocationType:
    print(f"  LocationType.{lt.name}")

Available Markets:
  Market.REAL_TIME_SCED
  Market.REAL_TIME_15_MIN
  Market.DAY_AHEAD_HOURLY

Available Location Types:
  LocationType.LOAD_ZONE
  LocationType.TRADING_HUB
  LocationType.RESOURCE_NODE
  LocationType.ELECTRICAL_BUS


## Settlement Point Prices (SPP)

Get real-time or day-ahead settlement point prices with optional filtering.

In [42]:
# Real-time 15-minute SPP
df = ercot.get_spp(
    start="2025-12-29T16:39",
    market=Market.REAL_TIME_15_MIN,
    location_type=LocationType.RESOURCE_NODE,
    locations=["7RNCHSLR_ALL"],
)

print(f"Real-Time SPP: {len(df):,} records")
df

Real-Time SPP: 42 records


,Time,End Time,Location,Price,Market,Location Type
0,2025-12-29 00:00:00-06:00,2025-12-29 00:15:00-06:00,7RNCHSLR_ALL,16.13,REAL_TIME_15_MIN,RN
1,2025-12-29 00:15:00-06:00,2025-12-29 00:30:00-06:00,7RNCHSLR_ALL,16.20,REAL_TIME_15_MIN,RN
2,2025-12-29 00:30:00-06:00,2025-12-29 00:45:00-06:00,7RNCHSLR_ALL,15.86,REAL_TIME_15_MIN,RN
3,2025-12-29 00:45:00-06:00,2025-12-29 01:00:00-06:00,7RNCHSLR_ALL,15.63,REAL_TIME_15_MIN,RN
4,2025-12-29 01:00:00-06:00,2025-12-29 01:15:00-06:00,7RNCHSLR_ALL,15.56,REAL_TIME_15_MIN,RN
5,2025-12-29 01:15:00-06:00,2025-12-29 01:30:00-06:00,7RNCHSLR_ALL,15.61,REAL_TIME_15_MIN,RN
6,2025-12-29 01:30:00-06:00,2025-12-29 01:45:00-06:00,7RNCHSLR_ALL,14.51,REAL_TIME_15_MIN,RN
7,2025-12-29 01:45:00-06:00,2025-12-29 02:00:00-06:00,7RNCHSLR_ALL,15.28,REAL_TIME_15_MIN,RN
8,2025-12-29 02:00:00-06:00,2025-12-29 02:15:00-06:00,7RNCHSLR_ALL,17.48,REAL_TIME_15_MIN,RN
9,2025-12-29 02:15:00-06:00,2025-12-29 02:30:00-06:00,7RNCHSLR_ALL,18.88,REAL_TIME_15_MIN,RN


In [35]:
spp_mapping =  ercot.get_settlement_point_mapping()
resource_nodes = spp_mapping["settlement_points"].RESOURCE_NODE.to_list()

for rn in resource_nodes:
    if str(rn).startswith("7RNCHSLR"):
        print(rn)

print(len(resource_nodes))


7RNCHSLR_ALL
18858


In [ ]:
# Filter to Load Zones only
df = ercot.get_spp(
    start="yesterday",
    market=Market.REAL_TIME_15_MIN,
    location_type=LocationType.LOAD_ZONE,
)


print(f"Load Zone SPP: {len(df):,} records")
df.head()

Load Zone SPP: 1,520 records


,Time,End Time,Location,Price,Market,Location Type
0,2025-12-28 23:30:00-06:00,2025-12-28 23:45:00-06:00,LZ_AEN,17.23,REAL_TIME_15_MIN,LZEW
1,2025-12-28 23:30:00-06:00,2025-12-28 23:45:00-06:00,LZ_AEN,17.22,REAL_TIME_15_MIN,LZ
2,2025-12-28 23:30:00-06:00,2025-12-28 23:45:00-06:00,LZ_CPS,16.92,REAL_TIME_15_MIN,LZ
3,2025-12-28 23:30:00-06:00,2025-12-28 23:45:00-06:00,LZ_CPS,16.93,REAL_TIME_15_MIN,LZEW
4,2025-12-28 23:30:00-06:00,2025-12-28 23:45:00-06:00,LZ_HOUSTON,17.47,REAL_TIME_15_MIN,LZEW


In [7]:
# Filter to specific locations
df = ercot.get_spp(
    start="yesterday",
    market=Market.REAL_TIME_15_MIN,
    locations=["LZ_HOUSTON", "LZ_NORTH", "HB_HOUSTON"],
)

print(f"Specific locations: {len(df):,} records")
df

Specific locations: 475 records


,Time,End Time,Location,Price,Market,Location Type
0,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,HB_HOUSTON,9.96,REAL_TIME_15_MIN,HU
1,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_HOUSTON,9.89,REAL_TIME_15_MIN,LZEW
2,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_HOUSTON,9.89,REAL_TIME_15_MIN,LZ
3,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_NORTH,9.87,REAL_TIME_15_MIN,LZ
4,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_NORTH,9.88,REAL_TIME_15_MIN,LZEW
...,...,...,...,...,...,...
470,2025-12-27 00:00:00-06:00,2025-12-27 00:15:00-06:00,HB_HOUSTON,13.94,REAL_TIME_15_MIN,HU
471,2025-12-27 00:00:00-06:00,2025-12-27 00:15:00-06:00,LZ_HOUSTON,13.93,REAL_TIME_15_MIN,LZ
472,2025-12-27 00:00:00-06:00,2025-12-27 00:15:00-06:00,LZ_HOUSTON,13.93,REAL_TIME_15_MIN,LZEW
473,2025-12-27 00:00:00-06:00,2025-12-27 00:15:00-06:00,LZ_NORTH,13.97,REAL_TIME_15_MIN,LZEW


In [8]:
# Day-Ahead SPP
df = ercot.get_spp(
    start="2025-12-28",
    market=Market.DAY_AHEAD_HOURLY,
    location_type=[LocationType.LOAD_ZONE, LocationType.TRADING_HUB],
)

print(f"Day-Ahead SPP: {len(df):,} records")
df

Day-Ahead SPP: 360 records


,Time,End Time,Location,Price,Market
0,2025-12-28 22:00:00-06:00,2025-12-28 23:00:00-06:00,LZ_AEN,15.20,DAY_AHEAD_HOURLY
1,2025-12-28 22:00:00-06:00,2025-12-28 23:00:00-06:00,LZ_CPS,15.22,DAY_AHEAD_HOURLY
2,2025-12-28 22:00:00-06:00,2025-12-28 23:00:00-06:00,LZ_HOUSTON,16.38,DAY_AHEAD_HOURLY
3,2025-12-28 22:00:00-06:00,2025-12-28 23:00:00-06:00,LZ_LCRA,15.19,DAY_AHEAD_HOURLY
4,2025-12-28 22:00:00-06:00,2025-12-28 23:00:00-06:00,LZ_NORTH,16.69,DAY_AHEAD_HOURLY
...,...,...,...,...,...
355,2025-12-28 03:00:00-06:00,2025-12-28 04:00:00-06:00,LZ_LCRA,6.52,DAY_AHEAD_HOURLY
356,2025-12-28 03:00:00-06:00,2025-12-28 04:00:00-06:00,LZ_NORTH,8.99,DAY_AHEAD_HOURLY
357,2025-12-28 03:00:00-06:00,2025-12-28 04:00:00-06:00,LZ_RAYBN,13.45,DAY_AHEAD_HOURLY
358,2025-12-28 03:00:00-06:00,2025-12-28 04:00:00-06:00,LZ_SOUTH,3.08,DAY_AHEAD_HOURLY


## Locational Marginal Prices (LMP)

Get LMP data by settlement point or electrical bus.

In [37]:
# Real-time LMP by settlement point
df = ercot.get_lmp(
    start="2025-12-29T09:55:00",
    market=Market.REAL_TIME_SCED,
)

print(f"Real-Time LMP: {len(df):,} records")
df.head()

Real-Time LMP: 60,000 records


,Location,Price,Market,SCED Time Stamp,Repeat Hour Flag
0,BASTEN_CCU,23.82,REAL_TIME_SCED,2025-12-29T10:25:16,False
1,BATCAVE_RN,27.28,REAL_TIME_SCED,2025-12-29T10:25:16,False
2,BAYC_BESS_RN,21.21,REAL_TIME_SCED,2025-12-29T10:25:16,False
3,BBREEZE_1_2,-12.35,REAL_TIME_SCED,2025-12-29T10:25:16,False
4,BCATWD_WD_1,14.50,REAL_TIME_SCED,2025-12-29T10:25:16,False


In [10]:
# LMP by electrical bus (more granular)
df = ercot.get_lmp(
    start="today",
    market=Market.REAL_TIME_SCED,
    location_type=LocationType.ELECTRICAL_BUS,
)

print(f"Electrical Bus LMP: {len(df):,} records")
df.head()

Electrical Bus LMP: 140,000 records


,Price,Market,SCED Time Stamp,Repeat Hour Flag,Electrical Bus
0,25.68,REAL_TIME_SCED,2025-12-28T17:25:15,False,ANNASW_8
1,26.35,REAL_TIME_SCED,2025-12-28T17:25:15,False,ANNASW_N5
2,25.68,REAL_TIME_SCED,2025-12-28T17:25:15,False,ANNASW_S8
3,25.88,REAL_TIME_SCED,2025-12-28T17:25:15,False,ANNA_RC_L_A
4,25.88,REAL_TIME_SCED,2025-12-28T17:25:15,False,ANNA_RC_L_B


In [ ]:
# Day-Ahead LMP
df = ercot.get_lmp(
    start="2025-12-29",
    market=Market.DAY_AHEAD_HOURLY,
)

print(f"Day-Ahead LMP: {len(df):,} records")
df.head()

Day-Ahead LMP: 192,592 records


,Time,End Time,Price,Market,Bus Name
0,2025-12-29 00:00:00-06:00,2025-12-29 01:00:00-06:00,18.14,DAY_AHEAD_HOURLY,_AK__AK_G1
1,2025-12-29 00:00:00-06:00,2025-12-29 01:00:00-06:00,18.16,DAY_AHEAD_HOURLY,_AZ_E_1
2,2025-12-29 00:00:00-06:00,2025-12-29 01:00:00-06:00,18.16,DAY_AHEAD_HOURLY,_AZ_L_D
3,2025-12-29 00:00:00-06:00,2025-12-29 01:00:00-06:00,18.15,DAY_AHEAD_HOURLY,_BI_138J
4,2025-12-29 00:00:00-06:00,2025-12-29 01:00:00-06:00,18.15,DAY_AHEAD_HOURLY,_BI_138L


## Ancillary Services

In [44]:
# Ancillary Service Prices (MCPC)
df = ercot.get_as_prices(start="yesterday")

print(f"AS Prices: {len(df):,} records")
df.head()

AS Prices: 120 records


,Time,End Time,Ancillary Type,MCPC
0,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,RRS,0.36
1,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,NSPIN,2.00
2,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,REGUP,0.57
3,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,REGDN,0.50
4,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,ECRS,0.45


In [45]:
# Ancillary Service Plan (requirements by hour)
df = ercot.get_as_plan(start="yesterday")

print(f"AS Plan: {len(df):,} records")
df.head()

AS Plan: 840 records


,Time,End Time,Posted,Ancillary Type,Quantity
0,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,2025-12-28T05:00:00,ECRS,864
1,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,2025-12-28T05:00:00,NSPIN,2278
2,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,2025-12-28T05:00:00,REGDN,315
3,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,2025-12-28T05:00:00,REGUP,369
4,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,2025-12-28T05:00:00,RRS,2982


## Shadow Prices

In [15]:
# SCED Shadow Prices (real-time)
df = ercot.get_shadow_prices(
    start="yesterday",
    market=Market.REAL_TIME_SCED,
)

print(f"SCED Shadow Prices: {len(df):,} records")
df.head()

SCED Shadow Prices: 2,011 records


,SCEDTimeStamp,Constraint ID,Constraint Name,Contingency Name,Shadow Price,Max Shadow Price,Limit,Value,Violated MW,From Station,To Station,From Station kV,To Station kV,CCT Status
0,12/27/2025 22:55:15,6,PNHNDL,BASE CASE,0.00000,5251,2369.3,2217.1,-152.2,NaN,NaN,0,0,NONCOMP
1,12/27/2025 22:55:15,9,BLESSI_LOLITA1_1,DSTEXP12,0.00000,3500,209.6,178.7,-31.0,LOLITA,BLESSING,138,138,COMP
2,12/27/2025 22:55:15,7,STPELM27_1,DSTEXP12,0.00000,4500,613.2,542.4,-70.8,ELMCREEK,STP,345,345,COMP
3,12/27/2025 22:55:15,2,587__A,MRNKDHM5,511.61374,3500,210.4,210.4,0.0,ARGYL,LWSVH,138,138,NONCOMP
4,12/27/2025 22:55:15,8,940__A,SPEBTRU8,0.00000,3500,250.7,217.3,-33.5,ENWSW,TMPTN,138,138,NONCOMP


In [16]:
# Day-Ahead Shadow Prices
df = ercot.get_shadow_prices(
    start="yesterday",
    market=Market.DAY_AHEAD_HOURLY,
)

print(f"DAM Shadow Prices: {len(df):,} records")
df.head()

DAM Shadow Prices: 730 records


,Time,End Time,Constraint Id,Constraint Name,Contingency Name,Constraint Limit,Constraint Value,Violation Amount,Shadow Price,From Station,To Station,From Station kV,To Station kV,Delivery Time
0,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,1,PNHNDL,BASE CASE,2480,2480,0,9.970,,,0.0,0.0,2025-12-27T01:00:00
1,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,2,DEC_G1NX,BASE CASE,70,70,0,0.010,DEC,DEC,13.8,1.0,2025-12-27T01:00:00
2,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,3,587__A,MRNKDHM5,220,220,0,218.951,ARGYL,LWSVH,138.0,138.0,2025-12-27T01:00:00
3,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,4,HARGRO_TWINBU1_1,DBAKCED5,151,151,0,14.268,TWINBU,HARGROVE,138.0,138.0,2025-12-27T01:00:00
4,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,5,6965__A,DBAKCED5,1331,1331,0,35.197,LNGSW,PRLSW,345.0,345.0,2025-12-27T01:00:00


## System Load

## Dashboard API (Placeholder Methods)

**Note:** These dashboard methods are placeholders. ERCOT does not provide documented public JSON endpoints for dashboard data. The methods return empty data or placeholder values.

**For real data, use the authenticated API methods instead:**
- System load: `get_actual_system_load_by_weather_zone()`
- Wind forecasts: `get_wpp_hourly_average_actual_forecast()` 
- Solar forecasts: `get_spp_hourly_average_actual_forecast()`
- Load forecasts: `get_load_forecast_by_weather_zone()`

The dashboard methods are kept as placeholders for potential future implementation.

In [17]:
# NOTE: Dashboard methods are placeholders - ERCOT doesn't provide public JSON endpoints
# Use authenticated API methods for real data instead

# Example of placeholder (returns default/empty values):
# status = ercot.get_status()  # Returns placeholder GridStatus
# print(status.message)  # "Dashboard data not available..."

# For real system data, use authenticated methods:
print("For real system data, use authenticated API methods:")
print("  - ercot.get_actual_system_load_by_weather_zone()")
print("  - ercot.get_wpp_hourly_average_actual_forecast()")
print("  - ercot.get_load_forecast_by_weather_zone()")

Failed to fetch grid status: Client error '404 404' for url 'https://www.ercot.com/api/1/services/read/dashboards/current-conditions.json'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404


Grid Condition: GridCondition.NORMAL
Current Load: 0 MW
Current Frequency: 60.000 Hz
Capacity: 0 MW
Reserves: 0 MW
Timestamp: 2025-12-28 17:34:18.615976-06:00


In [31]:
# Dashboard placeholder methods (return empty DataFrames):
# fuel_mix = ercot.get_fuel_mix()  # Placeholder
# esr = ercot.get_energy_storage_resources()  # Placeholder
# demand = ercot.get_system_wide_demand()  # Placeholder
# renewables = ercot.get_renewable_generation()  # Placeholder

# Instead, get real data with authenticated methods:
print("Real-time load data (using authenticated client):")
load = ercot.get_load(start="today", by="weather_zone")
print(f"Load data: {len(load)} records")
load.head()

Real-time load data (using authenticated client):
Load data: 0 records


,Operating Day,Hour Ending,Coast,East,Far West,North,NorthC,Southern,SouthC,West,Total,DST Flag


In [32]:
# Wind generation forecast (using authenticated client)
print("Wind forecast data:")
wind = ercot.get_wind_forecast(start="today")
print(f"Wind forecast: {len(wind)} records")
wind.head()

Wind forecast data:
Wind forecast: 408 records


,Time,End Time,Posted,Generation System Wide,COP HSL System Wide,STWPF System Wide,WGRPP System Wide,Generation Load Zone South Houston,COP HSL Load Zone South Houston,STWPF Load Zone South Houston,WGRPP Load Zone South Houston,Generation Load Zone West,COP HSL Load Zone West,STWPF Load Zone West,WGRPP Load Zone West,Generation Load Zone North,COP HSL Load Zone North,STWPF Load Zone North,WGRPP Load Zone North,HSL System Wide
0,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,2025-12-28T16:55:34,25025.58,27712.8,28230.0,27154.4,6415.52,6417.5,6662.4,6381.7,15841.16,18403.1,18666.2,17964.9,2768.90,2892.2,2901.4,2807.8,28274.42
1,2025-12-28 01:00:00-06:00,2025-12-28 02:00:00-06:00,2025-12-28T16:55:34,24285.22,26878.7,27595.4,26526.9,6497.87,6598.0,6851.8,6570.7,15195.79,17489.8,17991.0,17297.2,2591.56,2790.9,2752.6,2659.0,27598.33
2,2025-12-28 02:00:00-06:00,2025-12-28 03:00:00-06:00,2025-12-28T16:55:34,23177.99,25964.0,25630.5,24553.9,6241.89,6537.1,6542.2,6259.5,14482.14,16724.5,16598.9,15898.9,2453.96,2702.4,2489.4,2395.5,25735.23
3,2025-12-28 03:00:00-06:00,2025-12-28 04:00:00-06:00,2025-12-28T16:55:34,21172.84,24653.3,23138.9,22042.4,5738.08,6364.5,5862.6,5575.0,13090.12,15740.6,14908.9,14195.7,2344.64,2548.2,2367.4,2271.7,23158.66
4,2025-12-28 04:00:00-06:00,2025-12-28 05:00:00-06:00,2025-12-28T16:55:34,19155.73,22724.8,20717.8,19623.9,5491.21,5852.3,5520.9,5231.3,11380.17,14526.5,12985.7,12275.7,2284.35,2346.0,2211.2,2116.9,20704.17


In [33]:
# Solar generation forecast (using authenticated client)
print("Solar forecast data:")
solar = ercot.get_solar_forecast(start="today")
print(f"Solar forecast: {len(solar)} records")
solar.head()

Solar forecast data:
Solar forecast: 408 records


,Time,End Time,Posted,Generation System Wide,COP HSL System Wide,STPPF System Wide,PVGRPP System Wide,HSL System Wide
0,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,2025-12-28T16:55:34,0.39,0.0,0.0,0.0,0.85
1,2025-12-28 01:00:00-06:00,2025-12-28 02:00:00-06:00,2025-12-28T16:55:34,0.43,0.0,0.0,0.0,0.82
2,2025-12-28 02:00:00-06:00,2025-12-28 03:00:00-06:00,2025-12-28T16:55:34,0.43,0.0,0.0,0.0,0.85
3,2025-12-28 03:00:00-06:00,2025-12-28 04:00:00-06:00,2025-12-28T16:55:34,0.46,0.0,0.0,0.0,0.83
4,2025-12-28 04:00:00-06:00,2025-12-28 05:00:00-06:00,2025-12-28T16:55:34,0.44,0.0,0.0,0.0,0.83


In [34]:
# Load forecast (using authenticated client)
print("Load forecast data:")
forecast = ercot.get_load_forecast_by_weather_zone(
    start_date="2024-12-28",
    end_date="2024-12-29",
)
print(f"Load forecast: {len(forecast)} records")
forecast.head()

Load forecast data:
Load forecast: 63728 records


,Posted,Delivery Date,Hour Ending,Coast,East,Far West,North,North Central,South Central,Southern,West,System Total,Model,In Use Flag,DST Flag
0,2024-12-29T23:30:00,2024-12-29,1:00,10523.9043,1310.4789,7397.9399,1636.4709,10786.3496,5981.7266,3219.2007,1020.7552,41876.8261,A3,False,False
1,2024-12-29T23:30:00,2024-12-29,1:00,10511.6064,1281.8933,7408.4116,1594.2645,10893.6289,6074.6104,3259.3457,1007.2964,42031.0572,A6,False,False
2,2024-12-29T23:30:00,2024-12-29,1:00,10809.7002,1409.4301,7476.1401,1552.1899,11518.7998,6383.3301,3256.5200,1209.8500,43615.9602,E,False,False
3,2024-12-29T23:30:00,2024-12-29,1:00,10393.4004,1341.5400,7227.4600,1515.5699,10992.2998,6025.5801,3008.8601,1161.8500,41666.5603,E1,False,False
4,2024-12-29T23:30:00,2024-12-29,1:00,10601.2002,1345.9600,7231.8398,1517.6700,11275.5000,6146.1001,3021.6499,1178.8400,42318.7600,E2,False,False


## Historical Yearly Data (MIS Documents)

Access complete yearly historical data from ERCOT's Market Information System (MIS). These methods download and parse Excel/CSV files from ERCOT's document system.

**Available Methods:**
- `get_rtm_spp_historical(year)` - Full year of real-time settlement point prices
- `get_dam_spp_historical(year)` - Full year of day-ahead settlement point prices
- `get_settlement_point_mapping()` - Current settlement point to bus mapping

**Note:** These methods may take a few seconds as they download files from ERCOT's MIS system.

In [42]:
# Get historical RTM SPP for a specific year
# Note: This downloads data from ERCOT's MIS system and may take a few seconds
ercot_public = ERCOT()
rtm_2023 = ercot_public.get_rtm_spp_historical(2023)
print(f"RTM SPP 2023: {len(rtm_2023):,} records")
rtm_2023.head()

RTM SPP 2023: 68,448 records


,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point Name,Settlement Point Type,Settlement Point Price
0,01/01/2023,1,1,N,HB_BUSAVG,SH,-2.56
1,01/01/2023,1,2,N,HB_BUSAVG,SH,-2.34
2,01/01/2023,1,3,N,HB_BUSAVG,SH,-1.96
3,01/01/2023,1,4,N,HB_BUSAVG,SH,-1.60
4,01/01/2023,1,1,N,HB_HOUSTON,HU,-2.56


In [43]:
# Get settlement point mapping (returns dict of DataFrames)
mapping = ercot_public.get_settlement_point_mapping()
print(f"Settlement Point Mapping: {len(mapping)} tables")
for name, df in mapping.items():
    print(f"  {name}: {len(df):,} records")

# Show the main settlement points table
print("\nSettlement Points sample:")
mapping["settlement_points"].head()
mapping["noie"]

Settlement Point Mapping: 5 tables
  ccp: 68 records
  hubs: 11 records
  noie: 814 records
  resource_nodes: 1,568 records
  settlement_points: 18,858 records

Settlement Points sample:


,PHYSICAL_LOAD,NOIE,VOLTAGE_NAME,SUBSTATION,ELECTRICAL_BUS
0,AMD_T1X,LZ_AEN,138,ADVANCE,AMDX
1,AMD_T1Y,LZ_AEN,138,ADVANCE,AMDY
2,AG_123,LZ_AEN,138,ANGUS,ANGUSVAL
3,AG_456,LZ_AEN,138,ANGUS,ANGUSVALZ
4,AD_123,LZ_AEN,138,AUSTIN,AUSTIN_KN
...,...,...,...,...,...
809,WMFMWLD2,LZ_RAYBN,138,WMFMW_RC,EB2AAA
810,WMUNS_RC1,LZ_RAYBN,138,WMUNS_RC,WMUNS_RC_L_A
811,WMUNS_RC2,LZ_RAYBN,138,WMUNS_RC,WMUNS_RC_L_B
812,WYLIE_RC1,LZ_RAYBN,138,WYLIE_RC,WYLIE_RC_L_A


In [44]:
# Load by weather zone
df = ercot.get_load(start="today", by="weather_zone")

print(f"System Load: {len(df):,} records")
df

System Load: 0 records


,Operating Day,Hour Ending,Coast,East,Far West,North,NorthC,Southern,SouthC,West,Total,DST Flag


## Wind & Solar Forecasts

In [45]:
# Wind forecast
df = ercot.get_wind_forecast(start="2025-12-28")

print(f"Wind Forecast: {len(df):,} records")
df.tail(n=100)

Wind Forecast: 432 records


,Time,End Time,Posted,Generation System Wide,COP HSL System Wide,STWPF System Wide,WGRPP System Wide,Generation Load Zone South Houston,COP HSL Load Zone South Houston,STWPF Load Zone South Houston,WGRPP Load Zone South Houston,Generation Load Zone West,COP HSL Load Zone West,STWPF Load Zone West,WGRPP Load Zone West,Generation Load Zone North,COP HSL Load Zone North,STWPF Load Zone North,WGRPP Load Zone North,HSL System Wide
332,2025-12-28 20:00:00-06:00,2025-12-28 21:00:00-06:00,2025-12-28T04:55:33,NaN,24687.8,25246.2,23500.5,NaN,3172.1,3192.9,2738.6,NaN,18617.7,19155.3,18015.7,NaN,2898.0,2898.0,2746.2,NaN
333,2025-12-28 21:00:00-06:00,2025-12-28 22:00:00-06:00,2025-12-28T04:55:33,NaN,22505.5,23066.9,23066.9,NaN,2608.9,2619.4,2619.4,NaN,17123.3,17674.2,17674.2,NaN,2773.3,2773.3,2773.3,NaN
334,2025-12-28 22:00:00-06:00,2025-12-28 23:00:00-06:00,2025-12-28T04:55:33,NaN,21594.4,22144.9,22144.9,NaN,2380.0,2387.7,2387.7,NaN,16453.4,16996.2,16996.2,NaN,2761.0,2761.0,2761.0,NaN
335,2025-12-28 23:00:00-06:00,2025-12-29 00:00:00-06:00,2025-12-28T04:55:33,NaN,20320.5,20867.5,20867.5,NaN,2102.2,2107.9,2107.9,NaN,15512.4,16053.7,16053.7,NaN,2705.9,2705.9,2705.9,NaN
336,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,2025-12-28T03:55:34,25025.58,27712.8,28230.0,27154.4,6415.52,6417.5,6662.4,6381.7,15841.16,18403.1,18666.2,17964.9,2768.9,2892.2,2901.4,2807.8,28274.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,2025-12-28 19:00:00-06:00,2025-12-28 20:00:00-06:00,2025-12-28T00:55:35,NaN,24474.0,24935.3,23044.2,NaN,3193.6,3216.4,2724.8,NaN,18430.4,18866.0,17631.3,NaN,2850.0,2852.9,2688.1,NaN
428,2025-12-28 20:00:00-06:00,2025-12-28 21:00:00-06:00,2025-12-28T00:55:35,NaN,24743.0,25303.5,23416.4,NaN,3215.6,3239.7,2749.1,NaN,18640.6,19177.0,17944.9,NaN,2886.8,2886.8,2722.4,NaN
429,2025-12-28 21:00:00-06:00,2025-12-28 22:00:00-06:00,2025-12-28T00:55:35,NaN,22448.6,23010.9,23010.9,NaN,2609.0,2620.4,2620.4,NaN,17086.5,17637.4,17637.4,NaN,2753.1,2753.1,2753.1,NaN
430,2025-12-28 22:00:00-06:00,2025-12-28 23:00:00-06:00,2025-12-28T00:55:35,NaN,21588.1,22148.9,22148.9,NaN,2371.6,2380.2,2380.2,NaN,16463.3,17015.5,17015.5,NaN,2753.2,2753.2,2753.2,NaN


In [25]:
# Solar forecast
df = ercot.get_solar_forecast(start="yesterday")

print(f"Solar Forecast: {len(df):,} records")
df.head()

Solar Forecast: 576 records


,Time,End Time,Posted,Generation System Wide,COP HSL System Wide,STPPF System Wide,PVGRPP System Wide,HSL System Wide
0,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,2025-12-27T23:55:36,0.24,0.0,0.0,0.0,0.67
1,2025-12-27 01:00:00-06:00,2025-12-27 02:00:00-06:00,2025-12-27T23:55:36,0.29,0.0,0.0,0.0,0.81
2,2025-12-27 02:00:00-06:00,2025-12-27 03:00:00-06:00,2025-12-27T23:55:36,0.24,0.0,0.0,0.0,0.69
3,2025-12-27 03:00:00-06:00,2025-12-27 04:00:00-06:00,2025-12-27T23:55:36,0.26,0.0,0.0,0.0,0.67
4,2025-12-27 04:00:00-06:00,2025-12-27 05:00:00-06:00,2025-12-27T23:55:36,0.29,0.0,0.0,0.0,0.79


## Date Range Queries

Fetch data across multiple days.

In [46]:
# Get a week of data
df = ercot.get_spp(
    start="2024-12-20",
    end="2024-12-27",
    market=Market.DAY_AHEAD_HOURLY,
    location_type=LocationType.LOAD_ZONE,
)

print(f"Week of DAM SPP: {len(df):,} records")
df.head()

Week of DAM SPP: 1,152 records


,Time,End Time,Location,Price,Market
0,2024-12-26 00:00:00-06:00,2024-12-26 01:00:00-06:00,LZ_AEN,16.64,DAY_AHEAD_HOURLY
1,2024-12-26 00:00:00-06:00,2024-12-26 01:00:00-06:00,LZ_CPS,16.76,DAY_AHEAD_HOURLY
2,2024-12-26 00:00:00-06:00,2024-12-26 01:00:00-06:00,LZ_HOUSTON,16.72,DAY_AHEAD_HOURLY
3,2024-12-26 00:00:00-06:00,2024-12-26 01:00:00-06:00,LZ_LCRA,16.78,DAY_AHEAD_HOURLY
4,2024-12-26 00:00:00-06:00,2024-12-26 01:00:00-06:00,LZ_NORTH,16.75,DAY_AHEAD_HOURLY


## API Reference

### Unified API Methods

| Method | Description | Markets |
|--------|-------------|--------|
| `get_spp()` | Settlement Point Prices | REAL_TIME_15_MIN, DAY_AHEAD_HOURLY |
| `get_lmp()` | Locational Marginal Prices | REAL_TIME_SCED, DAY_AHEAD_HOURLY |
| `get_as_prices()` | Ancillary Service MCPC | - |
| `get_as_plan()` | AS Requirements | - |
| `get_shadow_prices()` | Transmission Constraints | REAL_TIME_SCED, DAY_AHEAD_HOURLY |
| `get_load()` | System Load by zone | - |
| `get_wind_forecast()` | Wind Generation Forecast | - |
| `get_solar_forecast()` | Solar Generation Forecast | - |

### Historical Yearly Methods (MIS Documents)

| Method | Description |
|--------|-------------|
| `get_rtm_spp_historical(year)` | Full year RTM SPP from MIS |
| `get_dam_spp_historical(year)` | Full year DAM SPP from MIS |
| `get_settlement_point_mapping()` | Settlement point to bus mapping |

### Direct Endpoint Methods (100+)

| Category | Example Methods |
|----------|----------------|
| Load | `get_actual_system_load_by_weather_zone()`, `get_load_forecast_by_weather_zone()` |
| Pricing | `get_dam_settlement_point_prices()`, `get_spp_node_zone_hub()` |
| Generation | `get_wpp_hourly_average_actual_forecast()`, `get_spp_hourly_average_actual_forecast()` |